In [1]:
import numpy as np
import matplotlib.mlab as mlab
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns

Goal: I want to calculate mortality rates by ethnicity and then compare this to the correlation between overall mortality and several sociodemographic factors.

In [2]:
df = pd.read_csv('COVID_Cases_Restricted_Detailed_10312020.csv')

In [39]:
df.head()

,race_ethnicity_combined,current_status,cdc_report_dt,sex,onset_dt,pos_spec_dt,hosp_yn,icu_yn,death_yn,hc_work_yn,pna_yn,abxchest_yn,acuterespdistress_yn,mechvent_yn,fever_yn,sfever_yn,chills_yn,myalgia_yn,runnose_yn,sthroat_yn,cough_yn,sob_yn,nauseavomit_yn,headache_yn,abdom_yn,diarrhea_yn,medcond_yn,county_fips_code,res_county,res_state,age_group
0,"White, Non-Hispanic",Laboratory-confirmed case,2020-10-01,Male,2020-08-26,NaN,Missing,Missing,Missing,Missing,Missing,Missing,Missing,Missing,Missing,Missing,Missing,Missing,Missing,Missing,Missing,Missing,Missing,Missing,Missing,Missing,Missing,31001.0,ADAMS,NE,10 - 19 Years
1,"White, Non-Hispanic",Laboratory-confirmed case,2020-08-23,Male,2020-07-07,NaN,Missing,Missing,Missing,Missing,Missing,Missing,Missing,Missing,Missing,Missing,Missing,Missing,Missing,Missing,Missing,Missing,Missing,Missing,Missing,Missing,Missing,31001.0,ADAMS,NE,10 - 19 Years
2,"White, Non-Hispanic",Laboratory-confirmed case,2020-10-01,Male,2020-09-02,NaN,Missing,Missing,Missing,Missing,Missing,Missing,Missing,Missing,Missing,Missing,Missing,Missing,Missing,Missing,Missing,Missing,Missing,Missing,Missing,Missing,Missing,31001.0,ADAMS,NE,10 - 19 Years
3,"White, Non-Hispanic",Laboratory-confirmed case,2020-10-01,Male,2020-09-01,NaN,Missing,Missing,Missing,Missing,Missing,Missing,Missing,Missing,Missing,Missing,Missing,Missing,Missing,Missing,Missing,Missing,Missing,Missing,Missing,Missing,Missing,31001.0,ADAMS,NE,10 - 19 Years
4,"White, Non-Hispanic",Laboratory-confirmed case,2020-09-18,Male,2020-09-01,NaN,Missing,Missing,Missing,Missing,Missing,Missing,Missing,Missing,Missing,Missing,Missing,Missing,Missing,Missing,Missing,Missing,Missing,Missing,Missing,Missing,Missing,31001.0,ADAMS,NE,10 - 19 Years


In [40]:
#Drop 'Missing' and 'Unknown' from death_yn so that there are only cases where death = Yes or No. Drop 'Unknown' cases from race_ethnicity
df_oct = df[(df.death_yn != 'Missing') & (df.death_yn != 'Unknown') & (df.race_ethnicity_combined != 'Unknown')]
df_oct['race_ethnicity_combined'].value_counts()

White, Non-Hispanic                                     923189
Hispanic/Latino                                         476914
Black, Non-Hispanic                                     316076
Multiple/Other, Non-Hispanic                             77836
Asian, Non-Hispanic                                      53659
American Indian/Alaska Native, Non-Hispanic               9689
Native Hawaiian/Other Pacific Islander, Non-Hispanic      4964
Name: race_ethnicity_combined, dtype: int64

In [41]:
#Abbreviate ethnicity_race names for simplicity
df_oct = df_oct.replace({'race_ethnicity_combined' : { 'White, Non-Hispanic' : 'W', 
                                                      'Hispanic/Latino' : 'H/L', 
                                                      'Black, Non-Hispanic' : 'B', 
                                                      'Multiple/Other, Non-Hispanic ' : 'M/O', 
                                                      'Asian, Non-Hispanic' : 'A', 
                                                      'American Indian/Alaska Native, Non-Hispanic' : 'AI/AN', 
                                                      'Native Hawaiian/Other Pacific Islander, Non-Hispanic' : 'NH/OPI'}})
df_oct['race_ethnicity_combined'].value_counts()

W         923189
H/L       476914
B         316076
M/O        77836
A          53659
AI/AN       9689
NH/OPI      4964
Name: race_ethnicity_combined, dtype: int64

In [42]:
#Determine % Mortality Rate by Ethnicity
W = df_oct[df_oct.race_ethnicity_combined == "W"]
W_Mortality_Rate = float(len(W[W.death_yn == 'Yes'])) / len(W)
H = df_oct[df_oct.race_ethnicity_combined == "H/L"]
H_Mortality_Rate = float(len(H[H.death_yn == 'Yes'])) / len(H)
B = df_oct[df_oct.race_ethnicity_combined == "B"]
B_Mortality_Rate = float(len(B[B.death_yn == 'Yes'])) / len(B)
M = df_oct[df_oct.race_ethnicity_combined == "M/O"]
M_Mortality_Rate = float(len(M[M.death_yn == 'Yes'])) / len(M)
A = df_oct[df_oct.race_ethnicity_combined == "A"]
A_Mortality_Rate = float(len(A[A.death_yn == 'Yes'])) / len(A)
AI = df_oct[df_oct.race_ethnicity_combined == "AI/AN"]
AI_Mortality_Rate = float(len(AI[AI.death_yn == 'Yes'])) / len(AI)
NH = df_oct[df_oct.race_ethnicity_combined == "NH/OPI"]
NH_Mortality_Rate = float(len(NH[NH.death_yn == 'Yes'])) / len(NH)

df_Mrate = pd.DataFrame([('W', W_Mortality_Rate*100), 
                         ('H/L', H_Mortality_Rate*100), 
                         ('B', B_Mortality_Rate*100), 
                         ('M/O', M_Mortality_Rate*100), 
                         ('A' , A_Mortality_Rate*100), 
                         ('AI/AN', AI_Mortality_Rate*100), 
                         ('NH/OPI', NH_Mortality_Rate*100)],
                  columns=['Ethnicity', '% Mortality Rate'])
df_Mrate

,Ethnicity,% Mortality Rate
0,W,7.716188
1,H/L,4.393035
2,B,8.279338
3,M/O,6.407061
4,A,9.129876
5,AI/AN,8.772835
6,NH/OPI,2.356970


Next step is to attach select sociodemographic factors to the CDC data so that the correlation with mortality can easily be calculated. The FIPS will be used to add the information by county.

In [43]:
df_oct.rename(columns={"county_fips_code": "FIPS"}, inplace=True)
print(df_oct.columns)

Index(['race_ethnicity_combined', 'current_status', 'cdc_report_dt', 'sex',
       'onset_dt', 'pos_spec_dt', 'hosp_yn', 'icu_yn', 'death_yn',
       'hc_work_yn', 'pna_yn', 'abxchest_yn', 'acuterespdistress_yn',
       'mechvent_yn', 'fever_yn', 'sfever_yn', 'chills_yn', 'myalgia_yn',
       'runnose_yn', 'sthroat_yn', 'cough_yn', 'sob_yn', 'nauseavomit_yn',
       'headache_yn', 'abdom_yn', 'diarrhea_yn', 'medcond_yn', 'FIPS',
       'res_county', 'res_state', 'age_group'],
      dtype='object')


In [44]:
#Load the table with sociodemographic data by county and rename the columns I want to use for better readability
df_health = pd.read_excel('Health Factors by County 2020 County Health Rankings Rows.xls', sheet_name='Ranked Measure Data')
df_health.rename(columns={'Adult obesity - % Adults with Obesity':'% Obesity'
                          , 'Adult smoking - % Smokers':'% Smokers', 
                          'Physical inactivity - % Physically Inactive':'% Phys. Inactive', 
                          'Uninsured - % Uninsured': '% Uninsured', 
                          'High school graduation - High School Graduation Rate':'% High School', 
                          'Some college - % Some College':'% Some College', 
                          'Unemployment - % Unemployed':'% Unemployed'}, inplace=True)
pd.set_option('display.max_columns', None)
df_health.head(3)

,FIPS,State,County,Premature death - Deaths,Premature death - Years of Potential Life Lost Rate,Premature death - 95% CI - Low,Premature death - 95% CI - High,Premature death - Quartile,Premature death - YPLL Rate (AIAN),Premature death - YPLL Rate (AIAN) 95% CI - Low,Premature death - YPLL Rate (AIAN) 95% CI - High,Premature death - YPLL Rate (Asian),Premature death - YPLL Rate (Asian) 95% CI - Low,Premature death - YPLL Rate (Asian) 95% CI - High,Premature death - YPLL Rate (Black),Premature death - YPLL Rate (Black) 95% CI - Low,Premature death - YPLL Rate (Black) 95% CI - High,Premature death - YPLL Rate (Hispanic),Premature death - YPLL Rate (Hispanic) 95% CI - Low,Premature death - YPLL Rate (Hispanic) 95% CI - High,Premature death - YPLL Rate (White),Premature death - YPLL Rate (White) 95% CI - Low,Premature death - YPLL Rate (White) 95% CI - High,Poor or fair health - % Fair or Poor Health,Poor or fair health - 95% CI - Low,Poor or fair health - 95% CI - High,Poor or fair health - Quartile,Poor physical health days - Average Number of Physically Unhealthy Days,Poor physical health days - 95% CI - Low,Poor physical health days - 95% CI - High,Poor physical health days - Quartile,Poor mental health days - Average Number of Mentally Unhealthy Days,Poor mental health days - 95% CI - Low,Poor mental health days - 95% CI - High,Poor mental health days - Quartile,Low birthweight - Unreliable,Low birthweight - % Low Birthweight,Low birthweight - 95% CI - Low,Low birthweight - 95% CI - High,Low birthweight - Quartile,Low birthweight - % LBW (AIAN),Low birthweight - % LBW (AIAN) 95% CI - Low,Low birthweight - % LBW (AIAN) 95% CI - High,Low birthweight - % LBW (Asian),Low birthweight - % LBW (Asian) 95% CI - Low,Low birthweight - % LBW (Asian) 95% CI - High,Low birthweight - % LBW (Black),Low birthweight - % LBW (Black) 95% CI - Low,Low birthweight - % LBW (Black) 95% CI - High,Low birthweight - % LBW (Hispanic),Low birthweight - % LBW (Hispanic) 95% CI - Low,Low birthweight - % LBW (Hispanic) 95% CI - High,Low birthweight - % LBW (White),Low birthweight - % LBW (White) 95% CI - Low,Low birthweight - % LBW (White) 95% CI - High,% Smokers,Adult smoking - 95% CI - Low,Adult smoking - 95% CI - High,Adult smoking - Quartile,% Obesity,Adult obesity - 95% CI - Low,Adult obesity - 95% CI - High,Adult obesity - Quartile,Food environment index - Food Environment Index,Food environment index - Quartile,% Phys. Inactive,Physical inactivity - 95% CI - Low,Physical inactivity - 95% CI - High,Physical inactivity - Quartile,Access to exercise opportunities - % With Access to Exercise Opportunities,Access to exercise opportunities - Quartile,Excessive drinking - % Excessive Drinking,Excessive drinking - 95% CI - Low,Excessive drinking - 95% CI - High,Excessive drinking - Quartile,Alcohol-impaired driving deaths - # Alcohol-Impaired Driving Deaths,Alcohol-impaired driving deaths - # Driving Deaths,Alcohol-impaired driving deaths - % Driving Deaths with Alcohol Involvement,Alcohol-impaired driving deaths - 95% CI - Low,Alcohol-impaired driving deaths - 95% CI - High,Alcohol-impaired driving deaths - Quartile,Sexually transmitted infections - # Chlamydia Cases,Sexually transmitted infections - Chlamydia Rate,Sexually transmitted infections - Quartile,Teen births - Teen Birth Rate,Teen births - 95% CI - Low,Teen births - 95% CI - High,Teen births - Quartile,Teen births - Teen Birth Rate (AIAN),Teen births - Teen Birth Rate (AIAN) 95% CI - Low,Teen births - Teen Birth Rate (AIAN) 95% CI - High,Teen births - Teen Birth Rate (Asian),Teen births - Teen Birth Rate (Asian) 95% CI - Low,Teen births - Teen Birth Rate (Asian) 95% CI - High,Teen births - Teen Birth Rate (Black),Teen births - Teen Birth Rate (Black) 95% CI - Low,Teen births - Teen Birth Rate (Black) 95% CI - High,Teen births - Teen Birth Rate (Hispanic),Teen births - Teen Birth Rate (Hispanic) 95% CI - Low,Teen births - Teen Birth Rate (Hispanic) 95% CI - High,Teen births - Teen Birth Rate (Wh

In [45]:
#Add the renamed columns on to the CDC data set based on the FIPS
df_oct = pd.merge(df_oct, df_health[['FIPS','% Obesity','% Smokers','% Phys. Inactive','% Uninsured','% High School','% Some College','% Unemployed']], on='FIPS', how='left')
df_oct.head(3)

,race_ethnicity_combined,current_status,cdc_report_dt,sex,onset_dt,pos_spec_dt,hosp_yn,icu_yn,death_yn,hc_work_yn,pna_yn,abxchest_yn,acuterespdistress_yn,mechvent_yn,fever_yn,sfever_yn,chills_yn,myalgia_yn,runnose_yn,sthroat_yn,cough_yn,sob_yn,nauseavomit_yn,headache_yn,abdom_yn,diarrhea_yn,medcond_yn,FIPS,res_county,res_state,age_group,% Obesity,% Smokers,% Phys. Inactive,% Uninsured,% High School,% Some College,% Unemployed
0,W,Laboratory-confirmed case,2020-08-31,Male,2020-08-22,NaN,Missing,No,No,No,No,No,No,No,No,Yes,Yes,Yes,No,No,No,No,Yes,Yes,No,No,Yes,39003.0,ALLEN,OH,40 - 49 Years,34.0,20.184418,27.5,7.696144,84.931507,61.94619,4.48647
1,W,Laboratory-confirmed case,2020-10-05,Male,2020-09-27,2020-09-29,No,No,No,No,No,No,No,No,Yes,Yes,Yes,No,Yes,Yes,Yes,No,No,Yes,No,No,No,39003.0,ALLEN,OH,40 - 49 Years,34.0,20.184418,27.5,7.696144,84.931507,61.94619,4.48647
2,W,Laboratory-confirmed case,2020-09-15,Male,2020-09-02,NaN,No,No,No,No,No,No,No,No,Yes,Yes,Yes,Yes,No,No,Yes,No,No,Yes,No,No,No,39003.0,ALLEN,OH,40 - 49 Years,34.0,20.184418,27.5,7.696144,84.931507,61.94619,4.48647


In [46]:
#Map death_yn to numeric binaries
df_oct['death_yn'] = df_oct['death_yn'].map({'No': 0,'Yes': 1})
df_oct.head(100)

,race_ethnicity_combined,current_status,cdc_report_dt,sex,onset_dt,pos_spec_dt,hosp_yn,icu_yn,death_yn,hc_work_yn,pna_yn,abxchest_yn,acuterespdistress_yn,mechvent_yn,fever_yn,sfever_yn,chills_yn,myalgia_yn,runnose_yn,sthroat_yn,cough_yn,sob_yn,nauseavomit_yn,headache_yn,abdom_yn,diarrhea_yn,medcond_yn,FIPS,res_county,res_state,age_group,% Obesity,% Smokers,% Phys. Inactive,% Uninsured,% High School,% Some College,% Unemployed
0,W,Laboratory-confirmed case,2020-08-31,Male,2020-08-22,NaN,Missing,No,0,No,No,No,No,No,No,Yes,Yes,Yes,No,No,No,No,Yes,Yes,No,No,Yes,39003.0,ALLEN,OH,40 - 49 Years,34.0,20.184418,27.5,7.696144,84.931507,61.946190,4.486470
1,W,Laboratory-confirmed case,2020-10-05,Male,2020-09-27,2020-09-29,No,No,0,No,No,No,No,No,Yes,Yes,Yes,No,Yes,Yes,Yes,No,No,Yes,No,No,No,39003.0,ALLEN,OH,40 - 49 Years,34.0,20.184418,27.5,7.696144,84.931507,61.946190,4.486470
2,W,Laboratory-confirmed case,2020-09-15,Male,2020-09-02,NaN,No,No,0,No,No,No,No,No,Yes,Yes,Yes,Yes,No,No,Yes,No,No,Yes,No,No,No,39003.0,ALLEN,OH,40 - 49 Years,34.0,20.184418,27.5,7.696144,84.931507,61.946190,4.486470
3,W,Laboratory-confirmed case,2020-07-06,Male,2020-04-12,NaN,Yes,No,0,No,No,Yes,No,No,Yes,No,No,No,No,No,Yes,Yes,No,No,No,No,Yes,39003.0,ALLEN,OH,40 - 49 Years,34.0,20.184418,27.5,7.696144,84.931507,61.946190,4.486470
4,W,Laboratory-confirmed case,2020-08-10,Male,2020-08-04,NaN,No,No,0,No,No,No,No,No,No,Unknown,Yes,No,Yes,Yes,Yes,No,No,Yes,No,No,Yes,39003.0,ALLEN,OH,40 - 49 Years,34.0,20.184418,27.5,7.696144,84.931507,61.946190,4.486470
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,W,Laboratory-confirmed case,2020-08-11,Male,2020-07-31,NaN,Yes,Missing,0,Missing,Missing,Missing,Missing,Missing,Missing,Missing,Missing,Missing,Missing,Missing,Missing,Missing,Missing,Missing,Missing,Missing,Missing,16001.0,Ada,ID,70 - 79 Years,25.6,11.990701,14.9,8.743292,84.494806,73.771048,2.459871
96,W,Laboratory-confirmed case,2020-08-05,Male,2020-08-02,NaN,No,Missing,0,Missing,Missing,Missing,Missing,Missing,Missing,Missing,Missing,Missing,Missing,Missing,Missing,Missing,Missing,Missing,Missing,Missing,Missing,16001.0,Ada,ID,70 - 79 Years,25.6,11.990701,14.9,8.743292,84.494806,73.771048,2.459871
97,W,Laboratory-confirmed case,2020-09-14,Male,2020-09-04,NaN,No,Missing,0,Missing,Missing,Missing,Missing,Missing,Missing,Missing,Missing,Missing,Missing,Missing,Missing,Missing,Missing,Missing,Missing,Missing,Missing,16001.0,Ada,ID,70 - 79 Years,25.6,11.990701,14.9,8.743292,84.494806,73.771048,2.459871
98,W,Laboratory-confirmed case,2020-09-02,Male,2020-08-30,NaN,No,Missing,0,Missing,Missing,Missing,Missing,Missing,Missing,Missing,Missing,Missing,Missing,Missing,Missing,Missing,Missing,Missing,Missing,Missing,Missing,16001.0,Ada,ID,70 - 79 Years,25.6,11.990701,14.9,8.743292,84.494806,73.771048,2.459871


In [47]:
from scipy import stats

In [48]:
#df_oct.dtypes

In [49]:
#uniqueValues = df_oct['% High School'].unique()
#print(uniqueValues)

In [50]:
#Drop nan values from added columns
df_oct.dropna(subset = ['% Obesity', '% Smokers', '% Phys. Inactive','% Uninsured',
                        '% High School','% Some College','% Unemployed'], inplace=True)

In [51]:
#Calculate regression between death_yn and socioeconomic factors
lin_reg1 = stats.linregress(x=df_oct["death_yn"], y=df_oct["% Obesity"])
lin_reg2 = stats.linregress(x=df_oct["death_yn"], y=df_oct["% Smokers"])
lin_reg3 = stats.linregress(x=df_oct["death_yn"], y=df_oct["% Phys. Inactive"])
lin_reg4 = stats.linregress(x=df_oct["death_yn"], y=df_oct["% Uninsured"])
lin_reg5 = stats.linregress(x=df_oct["death_yn"], y=df_oct["% High School"])
lin_reg6 = stats.linregress(x=df_oct["death_yn"], y=df_oct["% Some College"])
lin_reg7 = stats.linregress(x=df_oct["death_yn"], y=df_oct["% Unemployed"])
print(lin_reg1[2],";",lin_reg2[2],";",lin_reg3[2],";",lin_reg4[2],";",lin_reg5[2],";",lin_reg6[2],";",lin_reg7[2])

-0.08459854226592793 ; -0.06398524017613383 ; 0.014410677238303301 ; -0.0713606770161389 ; -0.049987534422671315 ; 0.025242602884312675 ; 0.0473538502896234
